In [ ]:
#April 4 Due date
#Xiying Hyper and Diya Hypo 
library(ISLR)
library(MASS)
library(tidyverse)
library(glmnet)
#library(leaps)

In [ ]:
options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)

In [ ]:
col_names_hyper <- c("Age", "Sex", "On_thyroxine", "Query_on_thyroxine", "On_anti_medi", "Sick", "Pregnant",
                     "Surgery", "I131_treatment", "Query_hypo", "Query_hyper", "Lithium", "Goitre", "Tumor",
                     "Hypopituitary", "Psych", "TSH_measured", "TSH", "T3_measured", "T3", "TT4_measured",
                     "TT4", "T4U_measured", "T4U", "FTI_measured", "FTI", "TBG_measured", "TBG", "Source", "Hyper")

In [ ]:
# Column names are from file allhyper.names
hyper <- read.table("allhyper.data", sep = ",")
hyper <- hyper %>%
    rename("Age"="V1", "Sex"="V2", "On_thyroxine"="V3", "Query_on_thyroxine"="V4", "On_anti_medi"="V5", "Sick"="V6", "Pregnant"="V7", "Surgery"="V8", "I131_treatment"="V9", "Query_hypo"="V10",
           "Query_hyper"="V11", "Lithium"="V12", "Goitre"="V13", "Tumor"="V14", "Hypopituitary"="V15", "Psych"="V16", "TSH_measured"="V17", "TSH"="V18", "T3_measured"="V19", "T3"="V20",
           "TT4_measured"="V21", "TT4"="V22", "T4U_measured"="V23", "T4U"="V24", "FTI_measured"="V25", "FTI"="V26", "TBG_measured"="V27", "TBG"="V28", "Source"="V29", "Hyper"="V30")
head(hyper)

,Age,Sex,On_thyroxine,Query_on_thyroxine,On_anti_medi,Sick,Pregnant,Surgery,I131_treatment,Query_hypo,Query_hyper,Lithium,Goitre,Tumor,Hypopituitary,Psych,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,Source,Hyper
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,41,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,?,SVHC,negative.|3733
2,23,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,?,f,?,f,?,other,negative.|1442
3,46,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,?,t,109,t,0.91,t,120,f,?,other,negative.|2965
4,70,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,?,f,?,f,?,other,negative.|806
5,70,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,?,SVI,negative.|2807
6,18,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.03,f,?,t,183,t,1.3,t,141,f,?,other,negative.|3434


In [ ]:
# Files with .data are training sets (2800 obs), and files with .test are testing sets (972 obs)
train <- hyper
str(train)

'data.frame':	2800 obs. of  30 variables:
 $ Age               : chr  "41" "23" "46" "70" ...
 $ Sex               : chr  "F" "F" "M" "F" ...
 $ On_thyroxine      : chr  "f" "f" "f" "t" ...
 $ Query_on_thyroxine: chr  "f" "f" "f" "f" ...
 $ On_anti_medi      : chr  "f" "f" "f" "f" ...
 $ Sick              : chr  "f" "f" "f" "f" ...
 $ Pregnant          : chr  "f" "f" "f" "f" ...
 $ Surgery           : chr  "f" "f" "f" "f" ...
 $ I131_treatment    : chr  "f" "f" "f" "f" ...
 $ Query_hypo        : chr  "f" "f" "f" "f" ...
 $ Query_hyper       : chr  "f" "f" "f" "f" ...
 $ Lithium           : chr  "f" "f" "f" "f" ...
 $ Goitre            : chr  "f" "f" "f" "f" ...
 $ Tumor             : chr  "f" "f" "f" "f" ...
 $ Hypopituitary     : chr  "f" "f" "f" "f" ...
 $ Psych             : chr  "f" "f" "f" "f" ...
 $ TSH_measured      : chr  "t" "t" "t" "t" ...
 $ TSH               : chr  "1.3" "4.1" "0.98" "0.16" ...
 $ T3_measured       : chr  "t" "t" "f" "t" ...
 $ T3                : chr  "2.5

In [ ]:
test <- read.table("allhyper.test", sep = ",")
test <- test %>%
    rename("Age"="V1", "Sex"="V2", "On_thyroxine"="V3", "Query_on_thyroxine"="V4", "On_anti_medi"="V5", "Sick"="V6", "Pregnant"="V7", "Surgery"="V8", "I131_treatment"="V9", "Query_hypo"="V10",
           "Query_hyper"="V11", "Lithium"="V12", "Goitre"="V13", "Tumor"="V14", "Hypopituitary"="V15", "Psych"="V16", "TSH_measured"="V17", "TSH"="V18", "T3_measured"="V19", "T3"="V20",
           "TT4_measured"="V21", "TT4"="V22", "T4U_measured"="V23", "T4U"="V24", "FTI_measured"="V25", "FTI"="V26", "TBG_measured"="V27", "TBG"="V28", "Source"="V29", "Hyper"="V30")
head(test)
str(test)

,Age,Sex,On_thyroxine,Query_on_thyroxine,On_anti_medi,Sick,Pregnant,Surgery,I131_treatment,Query_hypo,Query_hyper,Lithium,Goitre,Tumor,Hypopituitary,Psych,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,Source,Hyper
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,35,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,?,f,?,f,?,f,?,f,?,f,?,other,negative.|219
2,63,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,3.5,t,2.5,t,108,t,0.96,t,113,f,?,SVI,negative.|2059
3,25,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.6,t,2.4,t,61,t,0.82,t,75,f,?,SVHD,negative.|399
4,53,F,f,f,f,f,f,f,f,t,f,f,f,f,f,f,t,0.25,t,2.1,t,145,t,1.03,t,141,f,?,other,negative.|1911
5,92,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.7,t,1.3,t,120,t,0.84,t,143,f,?,SVI,negative.|487
6,67,M,f,f,f,f,f,f,f,t,f,f,f,f,f,f,t,0.81,f,?,t,84,t,0.83,t,101,f,?,other,negative.|1234


'data.frame':	972 obs. of  30 variables:
 $ Age               : int  35 63 25 53 92 67 60 60 48 27 ...
 $ Sex               : chr  "F" "M" "F" "F" ...
 $ On_thyroxine      : chr  "f" "f" "f" "f" ...
 $ Query_on_thyroxine: chr  "f" "f" "f" "f" ...
 $ On_anti_medi      : chr  "f" "f" "f" "f" ...
 $ Sick              : chr  "f" "f" "f" "f" ...
 $ Pregnant          : chr  "f" "f" "f" "f" ...
 $ Surgery           : chr  "f" "f" "f" "f" ...
 $ I131_treatment    : chr  "f" "f" "f" "f" ...
 $ Query_hypo        : chr  "f" "f" "f" "t" ...
 $ Query_hyper       : chr  "f" "f" "f" "f" ...
 $ Lithium           : chr  "f" "f" "f" "f" ...
 $ Goitre            : chr  "f" "f" "f" "f" ...
 $ Tumor             : chr  "f" "f" "f" "f" ...
 $ Hypopituitary     : chr  "f" "f" "f" "f" ...
 $ Psych             : chr  "f" "f" "f" "f" ...
 $ TSH_measured      : chr  "f" "t" "t" "t" ...
 $ TSH               : chr  "?" "3.5" "4.6" "0.25" ...
 $ T3_measured       : chr  "f" "t" "t" "t" ...
 $ T3                : chr

In [ ]:
# By taking others' research as a reference, variable Source is irrelevant
train <- train %>%
    dplyr::select(-Source)
test <- test %>%
    dplyr::select(-Source)
str(train)
str(test)

'data.frame':	2800 obs. of  29 variables:
 $ Age               : chr  "41" "23" "46" "70" ...
 $ Sex               : chr  "F" "F" "M" "F" ...
 $ On_thyroxine      : chr  "f" "f" "f" "t" ...
 $ Query_on_thyroxine: chr  "f" "f" "f" "f" ...
 $ On_anti_medi      : chr  "f" "f" "f" "f" ...
 $ Sick              : chr  "f" "f" "f" "f" ...
 $ Pregnant          : chr  "f" "f" "f" "f" ...
 $ Surgery           : chr  "f" "f" "f" "f" ...
 $ I131_treatment    : chr  "f" "f" "f" "f" ...
 $ Query_hypo        : chr  "f" "f" "f" "f" ...
 $ Query_hyper       : chr  "f" "f" "f" "f" ...
 $ Lithium           : chr  "f" "f" "f" "f" ...
 $ Goitre            : chr  "f" "f" "f" "f" ...
 $ Tumor             : chr  "f" "f" "f" "f" ...
 $ Hypopituitary     : chr  "f" "f" "f" "f" ...
 $ Psych             : chr  "f" "f" "f" "f" ...
 $ TSH_measured      : chr  "t" "t" "t" "t" ...
 $ TSH               : chr  "1.3" "4.1" "0.98" "0.16" ...
 $ T3_measured       : chr  "t" "t" "f" "t" ...
 $ T3                : chr  "2.5

In [ ]:
# Hyper shows the classification, ".|" and numbers after this indexing the obs, meaningless, can be removed
for (i in 1:2800){
    text <- train$Hyper[i]
    rex <- "\\..*"
    train$Hyper[i] <- str_replace(text, rex, "")
}

In [ ]:
for (i in 1:972){
    text <- test$Hyper[i]
    rex <- "\\..*"
    test$Hyper[i] <- str_replace(text, rex, "")
}

In [ ]:
# See if any 
for (i in 2:29){
    print(unique(train[[i]]))
}

[1] "F" "M" "?"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "t" "f"
  [1] "1.3"   "4.1"   "0.98"  "0.16"  "0.72"  "0.03"  "?"     "2.2"   "0.6"  
 [10] "2.4"   "1.1"   "2.8"   "3.3"   "12"    "1.2"   "1.5"   "6"     "2.1"  
 [19] "0.1"   "0.8"   "1.9"   "3.1"   "0.2"   "13"    "0.3"   "0.035" "2.5"  
 [28] "0.5"   "1.7"   "7.3"   "1.8"   "0.26"  "45"    "5.4"   "0.99"  "0.25" 
 [37] "0.92"  "0.15"  "0.64"  "1"     "0.4"   "2"     "2.6"   "14.8"  "15"   
 [46] "19"    "0.02"  "3"     "2.9"   "3.2"   "9"     "1.6"   "4.3"   "0.005"
 [55] "0.31"  "0.61"  "0.05"  "7.8"   "160"   "0.025" "1.4"   "0.01"  "8.8"  
 [64] "151"   "0.04"  "3.9"   "9.4"   "2.7"   "2.3"   "0.94"  "0.045" "3.5"  
 [73] "0.88"  "0.08"  "4.5"   "0.68"  "0.7"   "0.67"  "27"    "6.1"   "0.75" 
 [82] "0.55"  "26"    "5.2"   "0.77"  "0.07"  "0.9"   "11.4"  "143"   "0.45" 
 [91] "0.57"  "0.65"  "0

In [ ]:
for (i in 2:29){
    print(unique(test[[i]]))
}

[1] "F" "M" "?"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f" "t"
[1] "f"
[1] "f" "t"
[1] "f" "t"
  [1] "?"     "3.5"   "4.6"   "0.25"  "0.7"   "0.81"  "1.2"   "27"    "2.8"  
 [10] "2.6"   "4.4"   "3.1"   "1.1"   "4.5"   "0.14"  "0.06"  "7.4"   "2.3"  
 [19] "2"     "5.8"   "0.78"  "4"     "1"     "0.52"  "0.4"   "0.3"   "0.1"  
 [28] "1.9"   "17"    "1.3"   "0.2"   "0.005" "0.015" "12"    "1.5"   "2.9"  
 [37] "0.05"  "0.42"  "1.4"   "5"     "0.5"   "0.9"   "2.1"   "0.15"  "1.8"  
 [46] "7.6"   "1.6"   "0.79"  "3.7"   "0.77"  "0.64"  "2.7"   "0.98"  "0.025"
 [55] "0.93"  "0.58"  "0.34"  "3.4"   "0.86"  "0.47"  "1.7"   "0.95"  "34"   
 [64] "14.4"  "4.8"   "0.82"  "28"    "0.44"  "2.4"   "3.2"   "58"    "0.74" 
 [73] "0.01"  "0.67"  "21"    "0.8"   "0.02"  "165"   "4.2"   "8.9"   "7"    
 [82] "4.7"   "5.4"   "0.45"  "3"     "0.03"  "0.83"  "0.84"  "4.9"   "0.63" 
 [91] "3.3"   "2.5"   "9.2" 

In [ ]:
# We don't have any obs in variables TBG_measured and TBG, dropped
train <- train %>%
    select(-c(TBG_measured, TBG))
str(train)

'data.frame':	2800 obs. of  27 variables:
 $ Age               : chr  "41" "23" "46" "70" ...
 $ Sex               : chr  "F" "F" "M" "F" ...
 $ On_thyroxine      : chr  "f" "f" "f" "t" ...
 $ Query_on_thyroxine: chr  "f" "f" "f" "f" ...
 $ On_anti_medi      : chr  "f" "f" "f" "f" ...
 $ Sick              : chr  "f" "f" "f" "f" ...
 $ Pregnant          : chr  "f" "f" "f" "f" ...
 $ Surgery           : chr  "f" "f" "f" "f" ...
 $ I131_treatment    : chr  "f" "f" "f" "f" ...
 $ Query_hypo        : chr  "f" "f" "f" "f" ...
 $ Query_hyper       : chr  "f" "f" "f" "f" ...
 $ Lithium           : chr  "f" "f" "f" "f" ...
 $ Goitre            : chr  "f" "f" "f" "f" ...
 $ Tumor             : chr  "f" "f" "f" "f" ...
 $ Hypopituitary     : chr  "f" "f" "f" "f" ...
 $ Psych             : chr  "f" "f" "f" "f" ...
 $ TSH_measured      : chr  "t" "t" "t" "t" ...
 $ TSH               : chr  "1.3" "4.1" "0.98" "0.16" ...
 $ T3_measured       : chr  "t" "t" "f" "t" ...
 $ T3                : chr  "2.5

In [ ]:
test <- test %>%
    select(-c(TBG_measured, TBG))
str(test)

'data.frame':	972 obs. of  27 variables:
 $ Age               : int  35 63 25 53 92 67 60 60 48 27 ...
 $ Sex               : chr  "F" "M" "F" "F" ...
 $ On_thyroxine      : chr  "f" "f" "f" "f" ...
 $ Query_on_thyroxine: chr  "f" "f" "f" "f" ...
 $ On_anti_medi      : chr  "f" "f" "f" "f" ...
 $ Sick              : chr  "f" "f" "f" "f" ...
 $ Pregnant          : chr  "f" "f" "f" "f" ...
 $ Surgery           : chr  "f" "f" "f" "f" ...
 $ I131_treatment    : chr  "f" "f" "f" "f" ...
 $ Query_hypo        : chr  "f" "f" "f" "t" ...
 $ Query_hyper       : chr  "f" "f" "f" "f" ...
 $ Lithium           : chr  "f" "f" "f" "f" ...
 $ Goitre            : chr  "f" "f" "f" "f" ...
 $ Tumor             : chr  "f" "f" "f" "f" ...
 $ Hypopituitary     : chr  "f" "f" "f" "f" ...
 $ Psych             : chr  "f" "f" "f" "f" ...
 $ TSH_measured      : chr  "f" "t" "t" "t" ...
 $ TSH               : chr  "?" "3.5" "4.6" "0.25" ...
 $ T3_measured       : chr  "f" "t" "t" "t" ...
 $ T3                : chr

In [ ]:
# Changing variables to dummies and assign NA to null values
for (i in 1:2800){
    for (j in 3:26){
        if (train[i,j] == "f"){
            train[i,j] = 0
        } else if (train[i,j] == "t"){
            train[i,j] = 1
        } else if (train[i,j] == "?"){
            train[i,j] = NA
        }
    }
}

In [ ]:
for (i in 1:972){
    for (j in 3:26){
        if (test[i,j] == "f"){
            test[i,j] = 0
        } else if (test[i,j] == "t"){
            test[i,j] = 1
        } else if (test[i,j] == "?"){
            test[i,j] = NA
        }
    }
}

In [ ]:
# Changing Sex to a dummy variable where 1 means female and 0 means male
for (i in 1:2800){
    if (train$Sex[i] == "F"){
        train$Sex[i] = 1
    } else if (train$Sex[i] == "M"){
        train$Sex[i] = 0
    }
}
train <- train %>%
    rename("Female" = "Sex")

In [ ]:
for (i in 1:972){
    if (test$Sex[i] == "F"){
        test$Sex[i] = 1
    } else if (test$Sex[i] == "M"){
        test$Sex[i] = 0
    }
}
test <- test %>%
    rename("Female" = "Sex")

In [ ]:
str(train)
str(test)

'data.frame':	2800 obs. of  27 variables:
 $ Age               : chr  "41" "23" "46" "70" ...
 $ Female            : chr  "1" "1" "0" "1" ...
 $ On_thyroxine      : chr  "0" "0" "0" "1" ...
 $ Query_on_thyroxine: chr  "0" "0" "0" "0" ...
 $ On_anti_medi      : chr  "0" "0" "0" "0" ...
 $ Sick              : chr  "0" "0" "0" "0" ...
 $ Pregnant          : chr  "0" "0" "0" "0" ...
 $ Surgery           : chr  "0" "0" "0" "0" ...
 $ I131_treatment    : chr  "0" "0" "0" "0" ...
 $ Query_hypo        : chr  "0" "0" "0" "0" ...
 $ Query_hyper       : chr  "0" "0" "0" "0" ...
 $ Lithium           : chr  "0" "0" "0" "0" ...
 $ Goitre            : chr  "0" "0" "0" "0" ...
 $ Tumor             : chr  "0" "0" "0" "0" ...
 $ Hypopituitary     : chr  "0" "0" "0" "0" ...
 $ Psych             : chr  "0" "0" "0" "0" ...
 $ TSH_measured      : chr  "1" "1" "1" "1" ...
 $ TSH               : chr  "1.3" "4.1" "0.98" "0.16" ...
 $ T3_measured       : chr  "1" "1" "0" "1" ...
 $ T3                : chr  "2.5

In [ ]:
# Making sure our conversion is successful
for (i in 2:27){
    print(unique(train[[i]]))
}

[1] "1" "0" "?"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "1" "0"
  [1] "1.3"   "4.1"   "0.98"  "0.16"  "0.72"  "0.03"  NA      "2.2"   "0.6"  
 [10] "2.4"   "1.1"   "2.8"   "3.3"   "12"    "1.2"   "1.5"   "6"     "2.1"  
 [19] "0.1"   "0.8"   "1.9"   "3.1"   "0.2"   "13"    "0.3"   "0.035" "2.5"  
 [28] "0.5"   "1.7"   "7.3"   "1.8"   "0.26"  "45"    "5.4"   "0.99"  "0.25" 
 [37] "0.92"  "0.15"  "0.64"  "1"     "0.4"   "2"     "2.6"   "14.8"  "15"   
 [46] "19"    "0.02"  "3"     "2.9"   "3.2"   "9"     "1.6"   "4.3"   "0.005"
 [55] "0.31"  "0.61"  "0.05"  "7.8"   "160"   "0.025" "1.4"   "0.01"  "8.8"  
 [64] "151"   "0.04"  "3.9"   "9.4"   "2.7"   "2.3"   "0.94"  "0.045" "3.5"  
 [73] "0.88"  "0.08"  "4.5"   "0.68"  "0.7"   "0.67"  "27"    "6.1"   "0.75" 
 [82] "0.55"  "26"    "5.2"   "0.77"  "0.07"  "0.9"   "11.4"  "143"   "0.45" 
 [91] "0.57"  "0.65"  "0

In [ ]:
for (i in 2:27){
    print(unique(test[[i]]))
}

[1] "1" "0" "?"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0" "1"
[1] "0"
[1] "0" "1"
[1] "0" "1"
  [1] NA      "3.5"   "4.6"   "0.25"  "0.7"   "0.81"  "1.2"   "27"    "2.8"  
 [10] "2.6"   "4.4"   "3.1"   "1.1"   "4.5"   "0.14"  "0.06"  "7.4"   "2.3"  
 [19] "2"     "5.8"   "0.78"  "4"     "1"     "0.52"  "0.4"   "0.3"   "0.1"  
 [28] "1.9"   "17"    "1.3"   "0.2"   "0.005" "0.015" "12"    "1.5"   "2.9"  
 [37] "0.05"  "0.42"  "1.4"   "5"     "0.5"   "0.9"   "2.1"   "0.15"  "1.8"  
 [46] "7.6"   "1.6"   "0.79"  "3.7"   "0.77"  "0.64"  "2.7"   "0.98"  "0.025"
 [55] "0.93"  "0.58"  "0.34"  "3.4"   "0.86"  "0.47"  "1.7"   "0.95"  "34"   
 [64] "14.4"  "4.8"   "0.82"  "28"    "0.44"  "2.4"   "3.2"   "58"    "0.74" 
 [73] "0.01"  "0.67"  "21"    "0.8"   "0.02"  "165"   "4.2"   "8.9"   "7"    
 [82] "4.7"   "5.4"   "0.45"  "3"     "0.03"  "0.83"  "0.84"  "4.9"   "0.63" 
 [91] "3.3"   "2.5"   "9.2" 

In [ ]:
# Changing dummy variables from characters to numeric
for (i in 1:26){
    train[[i]] <- as.numeric(train[[i]])
}

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [ ]:
for (i in 1:26){
    test[[i]] <- as.numeric(test[[i]])
}

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [ ]:
str(train)
str(test)

'data.frame':	2800 obs. of  27 variables:
 $ Age               : num  41 23 46 70 70 18 59 80 66 68 ...
 $ Female            : num  1 1 0 1 1 1 1 1 1 0 ...
 $ On_thyroxine      : num  0 0 0 1 0 1 0 0 0 0 ...
 $ Query_on_thyroxine: num  0 0 0 0 0 0 0 0 0 0 ...
 $ On_anti_medi      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Sick              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Pregnant          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Surgery           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ I131_treatment    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Query_hypo        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Query_hyper       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Lithium           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Goitre            : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Tumor             : num  0 0 0 0 0 0 0 0 1 0 ...
 $ Hypopituitary     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Psych             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ TSH_measured      : num  1 1 1 1 1 1 0 1 1 1 ...
 $ TSH               : num  1.3 4.1 0.98 0.16 0.72 0.03 NA 2.2 0

In [ ]:
# NA in TSH, T3, TT4, T4U, FTI occurs only when these are not measured (0)
number <- rep(0,26)
for (i in 1:26){
    number[i] <- sum(is.na(train[[i]]))
}
name <- c("Age", "Sex", "On_thyroxine", "Query_on_thyroxine", "On_anti_medi", "Sick", "Pregnant",
                     "Surgery", "I131_treatment", "Query_hypo", "Query_hyper", "Lithium", "Goitre", "Tumor",
                     "Hypopituitary", "Psych", "TSH_measured", "TSH", "T3_measured", "T3", "TT4_measured",
                     "TT4", "T4U_measured", "T4U", "FTI_measured", "FTI")
(train.nulls <- data.frame(Columns = name, Counts = number))

Columns,Counts
<chr>,<dbl>
Age,1
Sex,110
On_thyroxine,0
Query_on_thyroxine,0
On_anti_medi,0
Sick,0
Pregnant,0
Surgery,0
I131_treatment,0


In [ ]:
number <- rep(0,26)
for (i in 1:26){
    number[i] <- sum(is.na(test[[i]]))
}
name <- c("Age", "Sex", "On_thyroxine", "Query_on_thyroxine", "On_anti_medi", "Sick", "Pregnant",
                     "Surgery", "I131_treatment", "Query_hypo", "Query_hyper", "Lithium", "Goitre", "Tumor",
                     "Hypopituitary", "Psych", "TSH_measured", "TSH", "T3_measured", "T3", "TT4_measured",
                     "TT4", "T4U_measured", "T4U", "FTI_measured", "FTI")
(test.nulls <- data.frame(Columns = name, Counts = number))

Columns,Counts
<chr>,<dbl>
Age,0
Sex,40
On_thyroxine,0
Query_on_thyroxine,0
On_anti_medi,0
Sick,0
Pregnant,0
Surgery,0
I131_treatment,0


In [ ]:
unique(train$Hyper)

[1] "negative"     "hyperthyroid" "T3 toxic"     "goitre"

In [ ]:
hyper <- c("negative", "hyperthyroid", "T3 toxic", "goitre")
number <- rep(0, 4)
for (i in 1:4){
    n <- train %>%
        filter(train$Hyper == hyper[i]) %>%
        nrow()
    number[i] <- n
}
(train.classes <- data.frame(Class = hyper, Counts = number))

Class,Counts
<chr>,<dbl>
negative,2723
hyperthyroid,62
T3 toxic,8
goitre,7


In [ ]:
unique(test$Hyper)

[1] "negative"        "hyperthyroid"    "T3 toxic"        "secondary toxic"
[5] "goitre"

In [ ]:
hyper <- c("negative", "hyperthyroid", "T3 toxic", "secondary toxic", "goitre")
number <- rep(0, 5)
for (i in 1:5){
    n <- test %>%
        filter(test$Hyper == hyper[i]) %>%
        nrow()
    number[i] <- n
}
(test.classes <- data.frame(Class = hyper, Counts = number))

Class,Counts
<chr>,<dbl>
negative,947
hyperthyroid,17
T3 toxic,2
secondary toxic,1
goitre,5


In [ ]:
# T3 toxic and goitre are not the same as hyperthyroid, but they are closely related to hyperthyroid, so also
# encode them as 1. Now our class is one when the obs has hyperthyroid or diagnoses diseases related to/ might
# lead to hyperthyroid
for (i in 1:2800){
    if (train$Hyper[i] == "negative"){
        train$Hyper[i] = 0
    } else if (train$Hyper[i] != "negative"){
        train$Hyper[i] = 1
    }
}
train$Hyper <- as.factor(train$Hyper)
unique(train$Hyper)
str(train)

[1] 0 1
Levels: 0 1

'data.frame':	2800 obs. of  27 variables:
 $ Age               : num  41 23 46 70 70 18 59 80 66 68 ...
 $ Female            : num  1 1 0 1 1 1 1 1 1 0 ...
 $ On_thyroxine      : num  0 0 0 1 0 1 0 0 0 0 ...
 $ Query_on_thyroxine: num  0 0 0 0 0 0 0 0 0 0 ...
 $ On_anti_medi      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Sick              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Pregnant          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Surgery           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ I131_treatment    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Query_hypo        : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Query_hyper       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Lithium           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Goitre            : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Tumor             : num  0 0 0 0 0 0 0 0 1 0 ...
 $ Hypopituitary     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Psych             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ TSH_measured      : num  1 1 1 1 1 1 0 1 1 1 ...
 $ TSH               : num  1.3 4.1 0.98 0.16 0.72 0.03 NA 2.2 0

In [ ]:
for (i in 1:972){
    if (test$Hyper[i] == "negative"){
        test$Hyper[i] = 0
    } else if (test$Hyper[i] != "negative"){
        test$Hyper[i] = 1
    }
}
test$Hyper <- as.factor(test$Hyper)
unique(test$Hyper)
str(test)

[1] 0 1
Levels: 0 1

'data.frame':	972 obs. of  27 variables:
 $ Age               : num  35 63 25 53 92 67 60 60 48 27 ...
 $ Female            : num  1 0 1 1 1 0 1 1 1 1 ...
 $ On_thyroxine      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Query_on_thyroxine: num  0 0 0 0 0 0 0 0 0 0 ...
 $ On_anti_medi      : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Sick              : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Pregnant          : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Surgery           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ I131_treatment    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Query_hypo        : num  0 0 0 1 0 1 0 0 0 0 ...
 $ Query_hyper       : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Lithium           : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Goitre            : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Tumor             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Hypopituitary     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Psych             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ TSH_measured      : num  0 1 1 1 1 1 1 1 1 1 ...
 $ TSH               : num  NA 3.5 4.6 0.25 0.7 0.81 1.2 27 2.8 2

In [ ]:
# How many obs have null values in these variables with a classification of 1
# If the amount is not very large comparing to total number of class 1 in our data, probably we could drop
# them
measures <- c(18, 20, 22, 24, 26)
columns <- c("TSH", "T3", "TT4", "T4U", "FTI")
number <- rep(0, 26)
for (i in measures){
    n <- train %>%
        filter(is.na(train[[i]]) == TRUE & train$Hyper == 1) %>%
        nrow()
    number[i] <- n
}
counts <- rep(0, 5)
for (i in 1:5){
    counts[i] <- number[measures[i]]
}
(train.null <- data.frame(Columns = columns, Counts = counts))

train %>%
    filter(is.na(train[[24]]) == TRUE & is.na(train[[26]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[24]]) == TRUE & is.na(train[[18]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[24]]) == TRUE & is.na(train[[20]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[18]]) == TRUE & is.na(train[[20]]) == TRUE & train$Hyper == 1) %>%
    nrow()

# If choose to drop NA obs in these variables from the train set, the number of class 1 would reduce from
# 77 to 63 (lose 14 obs)

Columns,Counts
<chr>,<dbl>
TSH,8
T3,9
TT4,0
T4U,1
FTI,1


[1] 1

[1] 0

[1] 0

[1] 4

In [ ]:
measures <- c(18, 20, 22, 24, 26)
columns <- c("TSH", "T3", "TT4", "T4U", "FTI")
number <- rep(0, 26)
for (i in measures){
    n <- test %>%
        filter(is.na(test[[i]]) == TRUE & test$Hyper == 1) %>%
        nrow()
    number[i] <- n
}
counts <- rep(0, 5)
for (i in 1:5){
    counts[i] <- number[measures[i]]
}
(test.null <- data.frame(Columns = columns, Counts = counts))

test %>%
    filter(is.na(test[[18]]) == TRUE & is.na(test[[20]]) == TRUE & test$Hyper == 1) %>%
    nrow()

# If choose to drop NA obs in these variables from the test set, the number of class 1 would reduce from
# 25 to 17 (lose 8 obs)

Columns,Counts
<chr>,<dbl>
TSH,5
T3,4
TT4,0
T4U,0
FTI,0


[1] 1

In [ ]:
train %>%
    filter(is.na(train[[1]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[2]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[2]]) == TRUE & is.na(train[[18]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[2]]) == TRUE & is.na(train[[20]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[2]]) == TRUE & is.na(train[[24]]) == TRUE & train$Hyper == 1) %>%
    nrow()
train %>%
    filter(is.na(train[[2]]) == TRUE & is.na(train[[18]]) == TRUE & is.na(train[[20]]) == TRUE & train$Hyper == 1) %>%
    nrow()

# If drop all nulls, the number of train set class 1 obs would drop by 18 in total, from 77 to 59

[1] 0

[1] 5

[1] 1

[1] 1

[1] 0

[1] 1

In [ ]:
test %>%
    filter(is.na(test[[2]]) == TRUE & test$Hyper == 1) %>%
    nrow()
test %>%
    filter(is.na(test[[2]]) == TRUE & is.na(test[[18]]) == TRUE & test$Hyper == 1) %>%
    nrow()
test %>%
    filter(is.na(test[[2]]) == TRUE & is.na(test[[20]]) == TRUE & test$Hyper == 1) %>%
    nrow()
test %>%
    filter(is.na(test[[2]]) == TRUE & is.na(test[[18]]) == TRUE & is.na(test[[20]]) == TRUE & test$Hyper == 1) %>%
    nrow()

# If drop all nulls, the number of test set class 1 obs would drop by 9 in total, from 25 to 16

[1] 2

[1] 1

[1] 1

[1] 1

In [ ]:
write.csv(train, "hyper_train.csv", row.names = FALSE)
write.csv(test, "hyper_test.csv", row.names = FALSE)